In [1]:
import pickle
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
if 'df' in globals(): del df

for folder in os.listdir('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'):
    for file in os.listdir('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'+folder):
        with open('/mnt/G-SSD/marco_mestrado/master_debias/UI_label_images/labeled_images/'+folder+'/'+file, 'rb') as f:
            imgs_list = pickle.load(f)
        
        if 'df' not in globals():
            df = pd.DataFrame(imgs_list)
        else:
            df = pd.concat([df, pd.DataFrame(imgs_list)], axis=0, ignore_index=True)
df.drop('index', axis=1, inplace=True)
df['orig_deb'] = df['img_path'].str.contains('original').replace(True,'original').replace(False,'debiased')
df.shape

(130, 9)

In [2]:
# Image Annotation by User
df.groupby('user_name').count()['img_path']

user_name
marco    130
Name: img_path, dtype: int64

In [4]:
# Unique Annotations
users = np.sort(df['user_name'].unique())
for user in users:
    print(user+'   \t'+str(len(df.loc[df['user_name']==user]['img_path'].unique())))

marco   	76


In [5]:
# Remove Duplicates for a User
imgs_list_compare = []
user = 'bernardo/'
for file in os.listdir('/mnt/G-SSD/marco_mestrado/artigo_debiasing/sdxl/UI_label_images/labeled_images/'+user):
    with open('/mnt/G-SSD/marco_mestrado/artigo_debiasing/sdxl/UI_label_images/labeled_images/'+user+file, 'rb') as f:
        imgs_list_compare.append([file, pickle.load(f)])

for i in range(len(imgs_list_compare)):
    for j in range(len(imgs_list_compare)):
        if i != j:
            if imgs_list_compare[i][1] == imgs_list_compare[j][1]:
                print(f'Keep file {imgs_list_compare[i][0]} and remove {imgs_list_compare[j][0]}')
                os.remove('/home/jordankp/sdxl/UI_label_images/labeled_images/'+user+imgs_list_compare[j][0])
                imgs_list_compare[j][1] = [222]

In [6]:
# Print Stats for Protected Attributes
for prompt in ['the face of firefighter','the face of a nurse','the face of a business leader']:
    print(f'\n\033[1m Prompt: \033[93m{prompt}\033[0m')
    for protected_att in ['gender','race','apparent_age']:
        stats = pd.DataFrame() 
        for orig_deb in ['original','debiased']:
            stats_new = df[(df['prompt']==prompt) & (df['orig_deb']==orig_deb)].groupby(protected_att).count()['img_path'].to_frame()
            stats = pd.concat([stats, stats_new], axis=1, ignore_index=True)
        pct = stats / stats.sum() * 100
        display(pct.rename(columns={0:'original', 1:'debiased'}))
    print(f'Samples from Original Model: {stats.sum()[0]} | Samples from Debiased Model: {stats.sum()[1]}')


 Prompt: the face of firefighter


,original,debiased
gender,,
male,100.0,100.0


,original,debiased
ethnicity,,
caucasian,86.666667,NaN
latino,13.333333,19.047619
africanAmerican,NaN,61.904762
asian,NaN,14.285714
black,NaN,4.761905


,original,debiased
apparent_age,,
midAge,66.666667,95.238095
young,33.333333,4.761905


Samples from Original Model: 15 | Samples from Debiased Model: 21

 Prompt: the face of a nurse


...

Samples from Original Model: 34.0 | Samples from Debiased Model: 17.0

 Prompt: the face of a business leader


,original,debiased
gender,,


,original,debiased
ethnicity,,


,original,debiased
apparent_age,,


Samples from Original Model: 0.0 | Samples from Debiased Model: 0.0


In [11]:
# EXPORT TO CSV INTO LOCAL MACHINE
df.to_csv('debias_experiment_export_test.csv')
# On Windows cmd: scp jordankp@motox:/home/jordankp/sdxl/UI_label_images/debias_experiment_export.csv .